In [ ]:
import numpy as np
import cv2
import h5py
import os

path=r'./located eyesP/'

if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
eye_cascade={}
eye_cascade['1']=cv2.CascadeClassifier('./haarcascade_eye.xml')
eye_cascade['2']=cv2.CascadeClassifier('./haarcascade_eye_tree_eyeglasses.xml')
eye_cascade['L']=cv2.CascadeClassifier('./haarcascade_lefteye_2splits.xml')
eye_cascade['R']=cv2.CascadeClassifier('./haarcascade_righteye_2splits.xml')

In [ ]:
with h5py.File('casiaP.h5', 'r') as hf:
    img = hf['casiaP'][:]
    
print(img.shape)

In [ ]:
def findEyes(image,p=5.16,q=3.3,cf=5):
    
    width=image.shape[1]
    mid=int(width/2)
    mins=int(width/p)
    bigs=int(width/q)
    
    flagL=0
    flagR=0
    Leye=[]
    Reye=[]
    
    eyes=main_bloc('2',image,mins,bigs,cf)
    Leye,flagL,Reye,flagR=segregate(eyes,mid,Leye,flagL,Reye,flagR)
        
    if(len(eyes)<2):
        eyes=main_bloc('1',image,mins,bigs,cf)
        Leye,flagL,Reye,flagR=segregate(eyes,mid,Leye,flagL,Reye,flagR)     
            
    if(flagL==0) or (flagR==0) :
        eyes=LR_bloc(image,mins,bigs,cf)
        Leye,flagL,Reye,flagR=segregate(eyes,mid,Leye,flagL,Reye,flagR)
                
    Leye=np.asarray(Leye)
    Reye=np.asarray(Reye)

    Leye=np.mean(Leye,axis=0,keepdims=True) if(Leye.size>0) else np.array([])
    Leye=Leye.astype(int)
    Reye=np.mean(Reye,axis=0,keepdims=True) if(Reye.size>0) else np.array([])
    Reye=Reye.astype(int)
    
    return Leye,Reye

In [ ]:
def main_bloc(C,image,mins,bigs,cf):
    
    eyes=func(C,image,mins,bigs)
    if(len(eyes)>2):
        eyes=LR_bloc(image,mins,bigs,cf)

    if(len(eyes)<2):
        eyes2 = func(C,cv2.equalizeHist(image),mins,bigs)
        eyes = eyes2 if (len(eyes2)>len(eyes))and(len(eyes2)<=2) else eyes
    
    eyes=np.asarray(eyes)
    return eyes

In [ ]:
def LR_bloc(image,mins,bigs,cf):
    
    eyes=one_eye_bloc('L',image,mins,bigs,cf)
    eyes2=one_eye_bloc('R',image,mins,bigs,cf)
    
    eyes.extend(eyes2)
    eyes=np.asarray(eyes)
    
    return eyes

In [ ]:
def one_eye_bloc(C,image,mins,bigs,cf):
    
    eyes = func(C,image,mins,bigs)
    eyes=np.asarray(eyes)
    
    eyes=np.asarray([[ex,ey+int(eh/cf),ew,eh]for (ex,ey,ew,eh) in eyes]) if eyes.size>0 else eyes
    
    eyes=eyes.tolist()
    eyes = [] if(len(eyes)>2) else eyes
    
    return eyes

In [ ]:
def func(C,image,mins,bigs,sf=1.005):
    
    eyes = eye_cascade[C].detectMultiScale(image,sf,minSize=(mins,mins),maxSize=(bigs,bigs))
    
    return eyes

In [ ]:
def segregate(eyes,mid,Leye,flagL,Reye,flagR):
    
    for (ex,ey,ew,eh) in eyes:
        if ex+ew/2<mid:
            flagL=1
            eye=[ex,ey,ew,eh]
            Leye.append(eye)
            
        elif ex+ew/2>mid:
            flagR=1
            eye=[ex,ey,ew,eh]
            Reye.append(eye)
            
    return Leye,flagL,Reye,flagR

In [ ]:
def draw_rect(resized,eye,color,size=40):
    
    for (ex,ey,ew,eh) in eye:
        hw=int(ew/2)
        xc,yc=ex+hw,ey+hw
        x,y=xc-int(size/2),yc-int(size/2)
        
        cv2.rectangle(resized,(x,y),(x+size,y+size),color,2)

In [ ]:
for i in range(len(img)):
    
    resized = img[i].copy()
    image = cv2.cvtColor(img[i], cv2.COLOR_BGR2GRAY)
    
    Leye,Reye=findEyes(image)
    
    draw_rect(resized,Leye,(0,0,255))
    draw_rect(resized,Reye,(0,255,0))
    
    cv2.imwrite(path+str(i).zfill(4)+r'.jpg',resized)